In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import rasterio as rs
import seaborn as sns
import os

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers import Conv2D, MaxPool2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.utils.vis_utils import plot_model
from matplotlib import pyplot as plt
from rasterio.plot import reshape_as_image
from rasterio.enums import Resampling
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.utils import class_weight
from tqdm import tqdm
from keras_tqdm import TQDMNotebookCallback

seed = 0
np.random.seed(seed)
palette = sns.color_palette("coolwarm", 5)
tqdm.pandas()

In [ ]:
if not os.path.exists('../data/training.pkl'):  
    df = pd.read_csv('../data/training.csv', dtype={'id':str})
    
    initial_size = len(df)
    print('Initial size %s:' % initial_size)

    df = df[df['area'] > 0.15]
    df = df[df['w'] >= 45]
    df = df[df['h'] >= 45]

    filtered_size = len(df)
    print('After filtering size %s (%s):' % (filtered_size, filtered_size / initial_size))

    data_path = '../data/fallow_land_sentinel'
    out_shape=(4, 50, 50)
    resampling=Resampling.nearest 

    def calculate_band(row):
        raster = rs.open('%s/%s/%s/%s' % (data_path, row['researcher'], row['id'], row['file'])).read(
            out_shape=out_shape, resampling=resampling
        )
        image = reshape_as_image(raster)
        return image

    df['image'] = df.progress_apply(calculate_band, axis=1)
    df['global_id'] = df['researcher'] + df['id']
    
    df.to_pickle('../data/training.pkl')
else:
    df = pd.read_pickle('../data/training.pkl')
         
def calculate_correctness(row):
    image = row['image']
    return np.count_nonzero(image) / image.size
    
df['correctness'] = df.progress_apply(calculate_correctness, axis=1)
df = df[df['correctness'] >= 0.9]        
        
print('Number of processed tiles: %s' % len(df))                      
sns.countplot(x='grade', data=df, palette=palette)
df.head()

In [ ]:
global_ids = df['global_id'].unique()
train_ids, test_ids = train_test_split(global_ids, test_size=0.1, random_state=0)
train_ids, val_ids = train_test_split(train_ids, test_size=0.1, random_state=0)

train_df = df[df['global_id'].isin(train_ids)]
test_df = df[df['global_id'].isin(test_ids)]
val_df = df[df['global_id'].isin(val_ids)]

del df

pd.DataFrame({
    'type': ['train', 'test', 'val'], 
    'count': [len(train_df), len(test_df), len(val_df)]
})

In [ ]:
def create_model(image_edge_size=50, kernel_size=(3,3), pool_size=(2,2), filters=16):

    model = Sequential()
    
    model.add(Conv2D(filters, kernel_size, input_shape = (image_edge_size, image_edge_size, 4)))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(Conv2D(filters, kernel_size, use_bias=False))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(MaxPool2D(pool_size=pool_size)) 
    model.add(Dropout(rate=0.5))

    model.add(Conv2D(filters*2, kernel_size, use_bias=False))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(Conv2D(filters*2, kernel_size, use_bias=False))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(MaxPool2D(pool_size=pool_size))
    model.add(Dropout(rate=0.5))

    model.add(Conv2D(filters*4, kernel_size, use_bias=False))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(Conv2D(filters*4, kernel_size, use_bias=False))
    model.add(BatchNormalization())
    model.add(Activation('elu'))    
    model.add(MaxPool2D(pool_size=pool_size))
    model.add(Dropout(rate=0.5))
    
    model.add(Flatten())
    model.add(Dense(filters*8, use_bias=False))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(Dropout(rate=0.5))
    
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])    
    return model

model = create_model()
model.summary()

In [ ]:
X_train = np.array(train_df.image.tolist())
y_train = np.array(train_df.label.tolist())

X_test = np.array(test_df.image.tolist())
y_test = np.array(test_df.label.tolist())

X_val = np.array(val_df.image.tolist())
y_val = np.array(val_df.label.tolist())

del train_df
del test_df
del val_df

In [ ]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)

class_weights

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=0, restore_best_weights=True)
reducelr = ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=0, factor=0.1)
tqmn_callback = TQDMNotebookCallback()

history = model.fit(x=X_train, 
                    y=y_train, 
                    batch_size=32, 
                    epochs=24, 
                    verbose=0,
                    validation_data=(X_val, y_val),
                    class_weight=class_weights,
                    callbacks=[reducelr, early_stopping, tqmn_callback])

In [ ]:
y_pred = model.predict(x=X_test, verbose=0)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
auc_r = auc(false_positive_rate, true_positive_rate)
print('test auc: %f' % auc_r)

score = model.evaluate(X_test, y_test, verbose=0)
print('test loss:', score[0])
print('test accuracy:', score[1])

In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(false_positive_rate, true_positive_rate, label='area = {:.5f}'.format(auc_r))
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.title('roc curve')
plt.legend(loc='best')

plt.subplot(1, 3, 2)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

plt.subplot(1, 3, 3)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

plt.show()

In [ ]:
y_pred_cls = model.predict_classes(x=X_test, verbose=0)
cm = confusion_matrix(y_test, y_pred_cls)

pd.DataFrame(cm, 
             index=['not a fallow', 'fallow'], 
             columns= ['predicted not a fallow', 'predicted fallow'])

In [ ]:
model.save('../models/fallow_full.h5')